In [1]:
using ITensors

# To Do 

- Add comments explaining stuff
- combine swap and beamsplitter into one function
- Maybe not destroy the original MPO when taking a projection ✅
- Finally actually check if we are getting the excpected state (bell state of sorts)✅(we are)
- write out measuring the stabalisers
- think about how to add noise cavities


When with a simple just beamsplitter case, the partial trace multiplies in the probability of the projection into the density matrix
<br/> (Confirming what we already knew)

# Functions

In [2]:
function ITensors.op(::OpName"Ground" , ::SiteType"Qudit" , d::Int)
    mat = zeros(d, d)
    mat[1,1] = 1
    return mat
end

In [3]:
function ITensors.op(::OpName"Excite1" , ::SiteType"Qudit" , d::Int)
    mat = zeros(d, d)
    mat[2,2] = 1
    return mat
end

In [27]:
function ITensors.expect(MPO_i, sites_i)
    list_exp = []
    for i in 1:length(sites_i)
        append!(list_exp, tr(apply( op("N", sites_i[i]), MPO_i)))
    end
    return list_exp
end

In [28]:
function beamsplitter(MPO_i, index_1, index_2, site_list)

    op_1 = ((op("A",site_list[index_1]) * op("Adag",site_list[index_2])) + (op("A",site_list[index_2]) * op("Adag",site_list[index_1])))
    H_ = exp((-im/4) * pi * op_1)

    H2 = apply(H_, MPO_i ; cutoff)
    H3 = conj(swapprime( apply( H_, swapprime(conj(H2), 1,0); cutoff ), 1,0))

    MPO_i[:] = H3
    return nothing
end

beamsplitter (generic function with 1 method)

In [29]:
function swap(MPO_i, index_1, index_2, site_list)

    op_1 = ((op("A",site_list[index_1]) * op("Adag",site_list[index_2])) + (op("A",site_list[index_2]) * op("Adag",site_list[index_1])))
    H_ = exp((-im/2) * pi * op_1)

    H2 = apply(H_, MPO_i ; cutoff, maxdim)
    H3 = conj(swapprime( apply( H_, swapprime(conj(H2), 1,0); cutoff, maxdim ), 1,0))

    MPO_i[:] = H3
    return nothing
end

swap (generic function with 1 method)

# Main Code

In [49]:
N = 8
sites = siteinds("Qudit", N, dim=3)
cutoff = 1E-8
maxdim = 100

100

In [50]:
input_list = [  repeat(["Excite1",], trunc(Int, N/2)) ; repeat(["Ground",], trunc(Int, N/2)) ]


8-element Vector{String}:
 "Excite1"
 "Excite1"
 "Excite1"
 "Excite1"
 "Ground"
 "Ground"
 "Ground"
 "Ground"

In [149]:
rails😄 = MPO(sites, input_list)
rails🤪 = MPO(sites, [  repeat(["Ground",], trunc(Int, N/2)) ; repeat(["Excite1",], trunc(Int, N/2)) ])

MPO
[1] ((dim=3|id=697|"Qudit,Site,n=1")', (dim=3|id=697|"Qudit,Site,n=1"), (dim=1|id=854|"Link,l=1"))
[2] ((dim=3|id=125|"Qudit,Site,n=2")', (dim=3|id=125|"Qudit,Site,n=2"), (dim=1|id=821|"Link,l=2"), (dim=1|id=854|"Link,l=1"))
[3] ((dim=3|id=102|"Qudit,Site,n=3")', (dim=3|id=102|"Qudit,Site,n=3"), (dim=1|id=555|"Link,l=3"), (dim=1|id=821|"Link,l=2"))
[4] ((dim=3|id=451|"Qudit,Site,n=4")', (dim=3|id=451|"Qudit,Site,n=4"), (dim=1|id=672|"Link,l=4"), (dim=1|id=555|"Link,l=3"))
[5] ((dim=3|id=913|"Qudit,Site,n=5")', (dim=3|id=913|"Qudit,Site,n=5"), (dim=1|id=761|"Link,l=5"), (dim=1|id=672|"Link,l=4"))
[6] ((dim=3|id=823|"Qudit,Site,n=6")', (dim=3|id=823|"Qudit,Site,n=6"), (dim=1|id=884|"Link,l=6"), (dim=1|id=761|"Link,l=5"))
[7] ((dim=3|id=96|"Qudit,Site,n=7")', (dim=3|id=96|"Qudit,Site,n=7"), (dim=1|id=484|"Link,l=7"), (dim=1|id=884|"Link,l=6"))
[8] ((dim=3|id=767|"Qudit,Site,n=8")', (dim=3|id=767|"Qudit,Site,n=8"), (dim=1|id=484|"Link,l=7"))


In [150]:
expect((0.9*rails😄 + 0.1rails🤪), sites)

8-element Vector{Any}:
 0.9
 0.9
 0.9
 0.9
 0.1
 0.1
 0.1
 0.1

In [145]:
expect(rails😄 + rails🤪 , sites)

8-element Vector{Any}:
 2.0
 2.0
 2.0
 2.0
 0.0
 0.0
 0.0
 0.0

In [51]:
rails = MPO(sites, input_list)
expect(rails, sites)

8-element Vector{Any}:
 1.0
 1.0
 1.0
 1.0
 0.0
 0.0
 0.0
 0.0

In [52]:
beamsplitter(rails, 2, 6, sites)
normalize!(rails)
println(tr(rails))
beamsplitter(rails, 3, 7, sites)
normalize!(rails)
println(tr(rails))
beamsplitter(rails, 4, 8, sites)
normalize!(rails)
println(tr(rails))
beamsplitter(rails, 1, 5, sites)
normalize!(rails)
println(tr(rails))
#cutoff = 20


1.0 + 0.0im
0.9999999999999998 + 1.8169632348103397e-16im


0.9999999999999982 - 1.3671525712072973e-16im
0.9999999999999978

 + 1.5006352874974966e-16im


In [53]:
beamsplitter(rails, 5, 6, sites)
normalize!(rails)
println(tr(rails))
beamsplitter(rails, 7, 8, sites)
normalize!(rails)
println(tr(rails))
beamsplitter(rails, 5, 8, sites)
normalize!(rails)
println(tr(rails))
beamsplitter(rails, 6, 7, sites)
normalize!(rails)
println(tr(rails))

0.9999999999999961 + 5.173483756631972e-16im
0.9999999999999967

 + 2.089064435542515e-16im


0.9999999999999977 - 6.908616133495565e-16im
0.9999999999999977

 - 6.848952571757802e-16im


In [54]:
swap(rails, 6, 8, sites)
tr(rails)

0.9999999999999958 - 7.601258468329633e-16im

In [55]:
expect(rails, sites)

8-element Vector{Any}:
 0.49999999999999756 + 1.7086618990255425e-15im
 0.49999999999999983 - 2.9665435239763894e-16im
   0.499999999999998 - 2.1609130341488255e-15im
  0.4999999999999973 - 2.596139802751989e-16im
 0.49999999999999817 - 9.896315565777421e-16im
  0.4999999999999978 - 1.0683120365144473e-16im
 0.49999999999999717 - 6.319558775774343e-16im
                 0.5 - 2.268520505831791e-15im

# Checking BS works

In [56]:
el = [2,2,1,1]

V = ITensor(1.)
for j=8:-1:5
  V *= ( state(sites[j],el[j-4])) * prime(state(sites[j],el[j-4])) * rails[j]
end

tensor = rails[1]*rails[2]*rails[3] *rails[4] * V
sub_sites = sites[1:4]


4-element Vector{Index{Int64}}:
 (dim=3|id=697|"Qudit,Site,n=1")
 (dim=3|id=125|"Qudit,Site,n=2")
 (dim=3|id=102|"Qudit,Site,n=3")
 (dim=3|id=451|"Qudit,Site,n=4")

In [58]:
baby_MPO = MPO( tensor ,sub_sites)
println(tr(baby_MPO))
baby_MPO /= (tr(baby_MPO))
#normalize!(baby_MPO)
tr(baby_MPO)

0.031249999999999726 - 2.6611312111079014e-16im


1.0 - 1.821639842903674e-18im

In [59]:
expect(baby_MPO,sub_sites)

4-element Vector{Any}:
 0.4999999999999998 + 2.9863051220046367e-15im
  0.500000000000001 + 4.1456688776065435e-15im
                0.5 - 2.5119786659353158e-15im
  0.499999999999999 + 1.9030854459973667e-15im

In [65]:
tr(baby_MPO)

1.0 - 1.821639842903674e-18im

In [140]:
el_2 = [2,2,1,1]
el_3 = [1,1,2,2]

V_2 = ITensor(1.)
for j=1:4
  V_2 *= ( state(sub_sites[j],el_3[j])) * prime(state(sub_sites[j],el_2[j])) * baby_MPO[j]
end



In [141]:
expect(baby_MPO, sub_sites)

4-element Vector{Any}:
 0.4999999999999998 + 2.9863051220046367e-15im
  0.500000000000001 + 4.1456688776065435e-15im
                0.5 - 2.5119786659353158e-15im
  0.499999999999999 + 1.9030854459973667e-15im

In [142]:
@show V_2

V_2 = ITensor ord=0
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 0-dimensional
0.5000000000000046 + 4.633935013109655e-15im


ITensor ord=0
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

# Cavities beamsplitter

In [206]:
no_cav = 3
cav_sites = siteinds("Qudit", 2*no_cav , dim=3)

6-element Vector{Index{Int64}}:
 (dim=3|id=375|"Qudit,Site,n=1")
 (dim=3|id=732|"Qudit,Site,n=2")
 (dim=3|id=254|"Qudit,Site,n=3")
 (dim=3|id=551|"Qudit,Site,n=4")
 (dim=3|id=863|"Qudit,Site,n=5")
 (dim=3|id=78|"Qudit,Site,n=6")

In [207]:
mode_mpo = []
for i in 1:no_cav
    ground_list = repeat(["Ground",], 2*no_cav)
    ground_list[i] = "Excite1"
    ground_list[i+no_cav] = "Excite1"
    push!(mode_mpo, MPO(cav_sites, ground_list))
end

In [208]:
cav_MPO = 0.95*mode_mpo[1] + 0.04*mode_mpo[2] + 0.01*mode_mpo[3]

MPO
[1] ((dim=3|id=375|"Qudit,Site,n=1")', (dim=3|id=375|"Qudit,Site,n=1"), (dim=3|id=698|"Link,l=1"))
[2] ((dim=3|id=732|"Qudit,Site,n=2")', (dim=3|id=732|"Qudit,Site,n=2"), (dim=3|id=49|"Link,l=2"), (dim=3|id=698|"Link,l=1"))
[3] ((dim=3|id=254|"Qudit,Site,n=3")', (dim=3|id=254|"Qudit,Site,n=3"), (dim=3|id=246|"Link,l=3"), (dim=3|id=49|"Link,l=2"))
[4] ((dim=3|id=551|"Qudit,Site,n=4")', (dim=3|id=551|"Qudit,Site,n=4"), (dim=3|id=911|"Link,l=4"), (dim=3|id=246|"Link,l=3"))
[5] ((dim=3|id=863|"Qudit,Site,n=5")', (dim=3|id=863|"Qudit,Site,n=5"), (dim=2|id=468|"Link,l=5"), (dim=3|id=911|"Link,l=4"))
[6] ((dim=3|id=78|"Qudit,Site,n=6")', (dim=3|id=78|"Qudit,Site,n=6"), (dim=2|id=468|"Link,l=5"))


In [197]:
cav_MPO = mode_mpo[1]

MPO
[1] ((dim=3|id=148|"Qudit,Site,n=1")', (dim=3|id=148|"Qudit,Site,n=1"), (dim=1|id=110|"Link,l=1"))
[2] ((dim=3|id=386|"Qudit,Site,n=2")', (dim=3|id=386|"Qudit,Site,n=2"), (dim=1|id=110|"Link,l=1"))


In [209]:
expect(cav_MPO, cav_sites)

6-element Vector{Any}:
 0.9499999999999997
 0.04
 0.01
 0.9499999999999997
 0.04
 0.01

In [210]:
# Beamsplitter

for i in 1:no_cav
    beamsplitter(cav_MPO, i, i+no_cav, cav_sites)
end

In [212]:
el_2 = [3,1,1,1,1,1]
#el_3 = [1,1,2,2]

V_2 = ITensor(1.)
for j=1:no_cav*2
  V_2 *= ( state(cav_sites[j],el_2[j])) * prime(state(cav_sites[j],el_2[j])) * cav_MPO[j]
end

In [213]:
@show V_2

V_2 = ITensor ord=0
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 0-dimensional
0.47499999999999887 + 1.0953910847812693e-16im


ITensor ord=0
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

# Appendix

function beamsplitter(MPO_i, index_1, index_2, site_list)
    #op_1 = op("Id",site_list[1])
    #op_1 = sqrt(0.5)* ((op("A",site_list[index_1]) * op("Adag",site_list[index_2])) + (op("A",site_list[index_2]) * op("Adag",site_list[index_1])))
    op_1 = ((op("A",site_list[index_1]) * op("Adag",site_list[index_2])) + (op("A",site_list[index_2]) * op("Adag",site_list[index_1])))
    #op_1 *= op("Id", site_list[3])
    #op_1 *= op("Id", site_list[2])

    #for i in 2:length(site_list)
    #    if i == index_1 || i==index_2
    #        nothing
    #    else
    #        op_1 *= op("Id",site_list[i])
    #    end
    #end
    #op_2 = op("A",site_list[index_2]) * op("Adag",site_list[index_1])
    #for i in 1:length(site_list)
    #    if i == index_1 || i==index_2
    #        nothing
    #    else
    #        op_2 *= op("Id",site_list[i])
    #    end
    #end
    #os = op_1 + op_2
    H_bs = exp((-im/4) * pi * op_1)
    #term2 = MPO(H_bs, site_list)

    c = cutoff
    d = maxdim

    #MPO_i[:] = apply(apply(term2, MPO_i ; cutoff= c , maxdim = d ),term2 ; cutoff= c , maxdim= d)
    return op_1
end


#Notes; 
#1. Product still works with one common index
#1.1 Order of MPO/A matters
#2. Apply needs common index with zero prime at least
#3. A(0,1) and rails(0,x) works
#4. A(0,x) and rails(0,x) or A(0,x) and rails(0,1) dont work

#Apply is complicated

In [804]:
A = randomITensor(sites[5:8]) * 0
A[1, 1] = 1
rails_MPS = MPS(A ,sites[5:8] ; cutoff = cutoff, maxdim = maxdim )

A[1, 1] = 1.0


1.0

In [801]:
Nvals = expect(rails_MPS,"N")

8-element Vector{Float64}:
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0